# How to combine colormaps for SatPy?

## 1. `proplot` method (recommended)

Let's read the file first. You can check the basic tutorial [here](https://github.com/zxdawn/FY-4/blob/master/satpy/examples/FY4A_agri_introduction%28EN%29.ipynb).

In [1]:
import os, glob
from satpy.scene import Scene
from trollimage.colormap import greys, spectral

# set the config path
os.environ['PPP_CONFIG_DIR'] = '../satpy_config/'

# load FY4A filenames
filenames = glob.glob('../data/FY4A-_AGRI*4000M_V0001.HDF')

# create the scene object
scn = Scene(filenames, reader='agri_l1')
channel = 'C12'
scn.load([channel])

# resample it to interested region
lekima_scene = scn.resample('lekima_4km')

E:\miniconda3\envs\satpy\lib\site-packages\pyproj\crs.py:562: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj_string = self.to_proj4()
E:\miniconda3\envs\satpy\lib\site-packages\pyproj\crs.py:562: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj_string = self.to_proj4()
E:\miniconda3\envs\satpy\lib\site-packages\pyproj\crs.py:562: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj_string = self.to_proj4()


In [ ]:
import proplot as plot

img = lekima_scene[channel]
# get lon.lat from scene
lon, lat = img.attrs['area'].get_lonlats()

# set axs
f, axs = plot.subplots(proj='pcarree')
axs.format(labels=True,
           lonlines=10,
           latlines=5,
           lonlim=(lon.min(), lon.max()),
           latlim=(lat.min(), lat.max()),
           geogridlinewidth=0.5
           )

# create our own colormap
ir_br = plot.Colormap('spectral_r', 'grays',
                      ratios=(1, 1), name='ir_br',
#                       save=True,
                      )


m = axs.pcolor(lon, lat, img-273.15, cmap=ir_br, levels=256)
axs.colorbar(m, loc='r', label='Brightness Temperature ($^\circ$C)')
axs.format(title=img.attrs['long_name'])
# f.savefig('./figures/agri_C12_colormap_proplot.png')

<img src='./figures/agri_C12_colormap_proplot.png'>

## 2. `enhancement` method

Add the configuration to `$PPP_CONFIG_DIR/enhancements/<sencor>.yaml` first.

I will use `agri` as an example below:

Here's the content of `agri.yaml`:

```
enhancements:
  AGRI_C12:
    name: C12
    operations:
      - name: colorize
        method: &colorizefun !!python/name:satpy.enhancements.colorize ''
        kwargs:
          palettes:
            - {colors: spectral, min_value: 193.15, max_value: 253.149999}
            - {colors: greys, min_value: 253.15, max_value: 303.15}
```

Some explanations can be found in this [issue](https://github.com/pytroll/satpy/issues/459) of satpy repository.
I just mention some important things here:

1. The filename should be <sensor>.yaml in case you mess up something;
2. `standard_name` should be set correctly;
    > Note that if you removed the first `name:` part then the enhancement would be used for all datasets matching standard_name.
    > 
    > If you remove `standard_name` it would match for only datasets matching the exact name.
    > 
    > If you have both it must match both of those items.
    > 
    > You could accomplish the same thing in generic.yaml by specifying sensor: abi in addition to name and/or standard_name.

Now, let's read the data and apply our own colormap to it:

Let make the figure more beautiful by adding colorbar:

In [3]:
# set the corresponding colormap ticks range
greys.set_range(-40, 30)
spectral.set_range(-90, -40)

# set the right width and height for colorbar
height, width = lekima_scene[channel].shape
h_legend = int(height/20)
w_legend = int(width/2)

# save dataset to image
lekima_scene.save_datasets(base_dir = './figures/',
                           filename='{sensor}_{name}_colormap_enhancement.png',
                           compute = True,
                           datasets = [channel],
                           writer = 'simple_image',
                           decorate={'decorate': [
                                    {'scale': {'colormap': spectral, 'extend': False,
                                               'width': w_legend, 'height': h_legend,
                                               'tick_marks': 5, 'minor_tick_marks': 1,
                                               'cursor': [0, 0], 'bg':'white',
                                                # title settings
                                                'title':'Brightness Temperature',
                                                'fontsize': h_legend*10, 'align': [w_legend, 0],
                                    }},
                                    {'scale': {'colormap': greys, 'extend': False,
                                               'width': w_legend, 'height': h_legend,
                                               'cursor': [w_legend, 0],
                                    }},
                        ]
                     }
                 )

E:\miniconda3\envs\satpy\lib\site-packages\dask\core.py:119: RuntimeWarning: invalid value encountered in less_equal
  return func(*args2)
E:\miniconda3\envs\satpy\lib\site-packages\dask\core.py:119: RuntimeWarning: invalid value encountered in greater_equal
  return func(*args2)


If you got the TypeError when setting the title of colorbar:
```
TypeError: text() argument 2 must be Font, not None
```
Please fix this issue by adding "check for font object" in "title" section, as mentioned in this [pull](https://github.com/pytroll/pydecorate/pull/9).

<img src='./figures/agri_C12_colormap_enhancement.png'>